In [1]:
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
import os
import glob
import shutil
from IPython.display import clear_output

from utils import label_map_util
from utils import visualization_utils_color as vis_util
from IPython.display import display, Image
from joblib import Parallel, delayed

In [2]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = './model/frozen_inference_graph_face.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './protos/face_label_map.pbtxt'

NUM_CLASSES = 2

In [3]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [4]:
def display_cv_image(image, format='.jpg'):
    decoded_bytes = cv2.imencode(format, image)[1].tobytes()
    display(Image(data=decoded_bytes))

In [5]:
class TensoflowFaceDector(object):
    def __init__(self, PATH_TO_CKPT):
        """Tensorflow detector
        """

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')


        with self.detection_graph.as_default():
            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(graph=self.detection_graph, config=config)
            self.windowNotSet = True


    def run(self, image):
        """image: bgr image
        return (boxes, scores, classes, num_detections)
        """

        image_np = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')
        # Actual detection.
        (boxes, scores, classes, num_detections) = self.sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
    
        return (boxes, scores, classes, num_detections)

In [6]:
def detect_n_faces_and_copyfile(imageFile):
    image = cv2.imread(imageFile)
    faceDetectImage = image.copy()
    [h, w] = faceDetectImage.shape[:2]

    (boxes, scores, classes, num_detections) = tDetector.run(faceDetectImage)

    faceBoxes = vis_util.visualize_boxes_and_labels_on_image_array(
        faceDetectImage,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=4)
    
    personCount = len(faceBoxes)
    
    if personCount == 0:
        shutil.copyfile(imageFile, noPersonFolder + imageFile.split("\\")[-1])
    elif personCount == 1:
        shutil.copyfile(imageFile, onlyFolder + imageFile.split("\\")[-1])
        saveFacePath = onlyFolderFace
    elif personCount == 2:
        shutil.copyfile(imageFile, twoShotFolder + imageFile.split("\\")[-1])
        saveFacePath = twoShotFolderFace
    else:
        shutil.copyfile(imageFile, overThreeFolder + imageFile.split("\\")[-1])
        saveFacePath = overThreeFolderFace
        
    imageHeight, imageWidth = image.shape[:2]
        
    for index, box in enumerate(faceBoxes):
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (int(xmin * imageWidth), int(xmax * imageWidth), int(ymin * imageHeight), int(ymax * imageHeight))
        
        cropWidth = right - left
        cropHeight = bottom - top
        
        if cropHeight > cropWidth:
            diff  = (cropHeight - cropWidth) / 2
            if int(left - diff) < 0 or int(right + diff) > imageWidth:
                top = int(top + diff)
                bottom = int(bottom - diff)
            else:
                left = int(left - diff)
                right = int(right + diff)
        else:
            diff = (cropWidth - cropHeight) / 2
            if int(top - diff) < 0 or int(bottom + diff) > imageHeight:
                left = int(left + diff)
                right = int(right - diff)
            else:
                top = int(top - diff)
                bottom = int(bottom + diff)
            
        cv2.imwrite(saveFacePath + imageFile.split("\\")[-1].split(".")[0] + "_face_" + str(index) + ".jpg", image[top:bottom, left:right])

In [7]:
tDetector = TensoflowFaceDector(PATH_TO_CKPT)

In [8]:
originFolderPath = "../images/origin/"
saveFolderPath = "../images/classification/"

In [9]:
for memberName in os.listdir(originFolderPath):

    # 必要なフォルダを作成
    onlyFolder = saveFolderPath + memberName + "/1_only/"
    twoShotFolder = saveFolderPath + memberName + "/2_shot/"
    overThreeFolder = saveFolderPath + memberName + "/3_over/"
    noPersonFolder = saveFolderPath + memberName + "/others/"
    onlyFolderFace = saveFolderPath + memberName + "/face/1_only/"
    twoShotFolderFace = saveFolderPath + memberName + "/face/2_shot/"
    overThreeFolderFace = saveFolderPath + memberName + "/face/3_over/"
    os.makedirs(onlyFolder, exist_ok=True)
    os.makedirs(twoShotFolder, exist_ok=True)
    os.makedirs(overThreeFolder, exist_ok=True)
    os.makedirs(noPersonFolder, exist_ok=True)
    os.makedirs(onlyFolderFace, exist_ok=True)
    os.makedirs(twoShotFolderFace, exist_ok=True)
    os.makedirs(overThreeFolderFace, exist_ok=True)
    
    fileList = glob.glob(originFolderPath + memberName + "/*")
    
    for imageFile in fileList:
        detect_n_faces_and_copyfile(imageFile)